Connect with your drive as we will need to clone the repository

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/GitHub/

/content/drive/MyDrive/GitHub


In [8]:
!git clone https://github.com/openai/mujoco-worldgen.git

Install all the dependancies for the env

In [3]:
!pip3 install -U 'mujoco-py<2.2,>=2.1'
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install free-mujoco-py

!pip install -r mujoco-worldgen/requirements.txt
!pip install -e mujoco-worldgen/
!pip install -e multi-agent-emergence-environments/

     |████████████████████████████████| 2.4 MB 5.2 MB/s 
     |████████████████████████████████| 206 kB 49.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 42 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
G

     |████████████████████████████████| 81 kB 3.9 MB/s 
     |████████████████████████████████| 25.2 MB 1.6 MB/s 
     |████████████████████████████████| 1.5 MB 42.9 MB/s 
     |████████████████████████████████| 127 kB 61.6 MB/s 
     |████████████████████████████████| 483 kB 56.0 MB/s 
     |████████████████████████████████| 120 kB 51.0 MB/s 
  Created wheel for gym: filename=gym-0.10.8-py3-none-any.whl size=1586965 sha256=aad83339f359f574e03ab3181351d66665f387116b5db252ddba3280e3db7b9a
  Stored in directory: /root/.cache/pip/wheels/c1/6f/02/d1cbfb00dae501539a2aa5c2bfb5ddc8018a0041fb92926315
  Created wheel for jsonnet: filename=jsonnet-0.11.2-cp37-cp37m-linux_x86_64.whl size=2896583 sha256=90cfc8ea086d0e58c8eb4633e4466c64b68cb22ab9823144633373fb17f35353
  Stored in directory: /root/.cache/pip/wheels/0c/a2/eb/17f92220a4d9ad6a8ffb90299a0fce2370c76ce864d4df74c2
  Created wheel for numpy-stl: filename=numpy_stl-2.10.1-cp37-cp37m-linux_x86_64.whl size=135390 sha256=bb1130ecde0fde00c35a812

Creation of the env

In [3]:
from mae_envs.envs import hide_and_seek

hs_env = hide_and_seek.make_env()

Compiling /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local/lib
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuexten

Understanding the env:

In [4]:
hs_env.reset()
print(hs_env.action_space)
print(hs_env.observation_space)

Dict(action_movement:Tuple(MultiDiscrete([11 11 11]), MultiDiscrete([11 11 11])), action_pull:Tuple(Discrete(2), Discrete(2)), action_glueall:Tuple(Discrete(2), Discrete(2)))
Dict(agent_qpos_qvel:Box(-inf, inf, (1, 10), float32), box_obs:Box(-inf, inf, (2, 12), float32), mask_aa_obs:Box(-inf, inf, (1,), float32), mask_ab_obs:Box(-inf, inf, (2,), float32), mask_ab_obs_spoof:Box(-inf, inf, (2,), float32), mask_ar_obs:Box(-inf, inf, (1,), float32), observation_self:Box(-inf, inf, (10,), float32), ramp_obs:Box(-inf, inf, (1, 12), float32))


Classes for both the types of agents in the env

In [5]:
class Hider:
  def __init__(self, env):
    self.env = env
    self.observation_space = env.observation_space
    self.action_space = env.action_space

  def step(self, observation):
    return np.random.choice(self.action_space.n)

class Seeker:
  def __init__(self, env):
    self.env = env
    self.observation_space = env.observation_space
    self.action_space = env.action_space

  def step(self, observation):
    return np.random.choice(self.action_space.n)

In [6]:
class Actor:
  def __init__(self, alfr, actor_input_state_dim, actor_output_action_dim):
    self.model = self.actor_network_model(alfr, actor_input_state_dim, actor_output_action_dim)
    self.opt = tf.keras.optimizers.Adam(0.0005)

  def actor_network_model(self, alr, actor_input_state_dim,actor_output_action_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=actor_input_state_dim, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actor_output_action_dim, activation='softmax'))
    return model

  def actor_train(self, obs, action, adv):
    with tf.GradientTape() as tape:
      actor_pred = self.model(obs, training=True)
      actor_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(action, actor_pred,sample_weight=tf.stop_gradient(adv))
    grads = tape.gradient(actor_loss, self.model.trainable_variables)
    self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
    return actor_loss

In [7]:
class Critic:
  def __init__(self, clr, critic_input_state_dim, critic_output_action_dim):
    self.model = self.critic_network_model(clr, critic_input_state_dim, critic_output_action_dim)
    self.opt = tf.keras.optimizers.Adam(0.0005)

  def critic_network_model(self,clr,critic_input_state_dim,critic_output_action_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=critic_input_state_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(critic_output_action_dim, activation='linear'))
    return model
    
  def critic_train(self, obs, adv):
    with tf.GradientTape() as tape:
      critic_pred = self.model(obs, training=True)
      critic_loss = tf.keras.losses.MeanSquaredError()(tf.stop_gradient(adv), critic_pred)
    critic_grads = tape.gradient(critic_loss, self.model.trainable_variables)
    self.opt.apply_gradients(zip(critic_grads, self.model.trainable_variables))
    return critic_loss

In [11]:
class A2C:
  def __init__(self, env, hider, seeker):
    self.env = env
    self.hider = hider
    self.seeker = seeker
    self.observation_space = env.observation_space     #dictionary with multiple keys
    self.obervation_size = len(env.observation_space) 
    obervation_size  
    self.action_space = env.action_space   #dictionary with 3 keys
    self.reward_per_episode = []
    self.average_reward = [0]*10

  def train(self,episodes,gamma,actor_lr, critic_lr):
    obs = self.env.reset()
    #action_size = self.env.action_space.shape[0]
    self.actor = Actor(actor_lr,self.obervation_size,action_size)
    self.critic = Critic(critic_lr,self.obervation_size,1)
    average_idx = 0

    #print("8: episodes begin")
    for eps_n in range(episodes):
        obs = self.env.reset()
        done_iter = False
        cumm_reward=0
        step_no=1

        #print("9: steps start for episode {}".format(str(eps_n)))
        while not done_iter:
          
          obs_reshaped = tf.reshape(helper(obs),(1,self.obervation_size))

          next_q = self.actor.model.predict(obs_reshaped)
          #print("next_q: {}".format(next_q))

          action_prob = tfp.distributions.Categorical(probs=next_q)
          #print("action_prob {}".format(action_prob))
          action = action_prob.sample()

          action_scaler = tf.gather(action,0).numpy()
          #print("action_scaler: {}".format(action_scaler))

          obs_next, reward, done_iter, info = self.env.step(action_scaler)
          #print("obs_next: {}".format(obs_next))
          #print("reward: {}".format(reward))
          #print("done_iter: {}".format(done_iter))
          #print("info: {}".format(info))

          critic_obs = self.critic.model.predict(obs_reshaped) 
          #print("critic_obs: {}".format(critic_obs))

          obs_next_reshaped = tf.reshape(helper(obs_next),(1,self.obervation_size))
          critic_obs_next = self.critic.model.predict(obs_next_reshaped) 
          #print("critic_obs_next: {}".format(critic_obs_next))

          adv = reward + (1-done_iter)*gamma*critic_obs_next - critic_obs
          #print("self.adv: {}".format(self.adv))

          critic_loss = self.critic.critic_train(obs_reshaped, adv+critic_obs)
          actor_loss = self.actor.actor_train(obs_reshaped,action, adv)

          cumm_reward += reward
          obs = obs_next
          step_no += 1

        self.reward_per_episode.append(cumm_reward)
        average_idx = 0 if average_idx == 9 else 1 + average_idx
        self.average_reward[average_idx] = cumm_reward
        avg_reward = int(sum(self.average_reward)/len(self.average_reward))
        if avg_reward >300: 
          break 
          print("average reward for the last consecutive episodes is "+ avg_reward +"greater than 470")
        print("cummulative reward is " + str(cumm_reward) + " for episode no. " + str(eps_n + 1)  )  

          
          

In [ ]:

hider = Hider(hs_env)
seeker = Seeker(hs_env)

number_of_episodes = 5000

a2c = A2C(hs_env, hider, seeker)

a2c.train(episodes = number_of_episodes,gamma = 0.99, actor_lr = 0.001, critic_lr = 0.001)